Import dataset for google colab

In [ ]:
## Run these cells at the first time of running project
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
# use this cell when you upload the dataset in your colab
!unzip -q "/content/dataset.zip" -d "/content/"


In [ ]:
## use this cell whean you connect your drive and load dataset from drive
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/MyDrive/Colab Notebooks/dataset.zip" -d "/content/drive/MyDrive/dataset/"


Config dataset addresses


In [ ]:
img_train = "/content/drive/MyDrive/dataset/csvTrainImages 60k x 784.csv"
label_train = "/content/drive/MyDrive/dataset/csvTrainLabel 60k x 1.csv"
img_test = "/content/drive/MyDrive/dataset/csvTestImages 10k x 784.csv"
label_test = "/content/drive/MyDrive/dataset/csvTestLabel 10k x 1.csv"

Load Dataset

In [ ]:
import numpy as np

images_train = np.loadtxt(open(img_train), delimiter=',')
label_train = np.loadtxt(open(label_train), delimiter=',')
images_test = np.loadtxt(open(img_test), delimiter=',')
label_test = np.loadtxt(open(label_test), delimiter=',')

# train images count is 60K (60K train image and 60K their labels)
print("Size of data loaded for train images is: {}".format(images_train.shape))
print("Size of data loaded for train labels is: {}".format(label_train.shape))
# test image count is 10K (10K test image and 10K their labels)
print("Size of data loaded for test images is: {}".format(images_test.shape))
print("Size of data loaded for test labels is: {}".format(label_test.shape))
